In [0]:
-- CONTRACT ANALYSIS
-- -- Doc Info

CREATE TABLE IF NOT EXISTS IDENTIFIER(:catalog || '.' || :schema || '.doc_info') (
  path STRING,
  agreement_name STRING,
  agreement_type STRING,
  document_type STRING,
  effective_date STRING,
  expiry_date STRING,
  is_master_agreement STRING,
  related_master_agreement_name STRING,
  amendment_expiry_date STRING,
  combined_doc_info STRING
)
USING DELTA;

MERGE INTO IDENTIFIER(:catalog || '.' || :schema || '.doc_info') AS target
USING (
  SELECT
    path,
    metadata.*,
    cast(to_variant_object(named_struct('path', path, 'metadata', metadata)) as string) AS combined_doc_info
  FROM (
    SELECT
      f.path,
      from_json(AI_QUERY(
        "databricks-claude-sonnet-4-5",
        SUBSTRING(CONCAT(
          'You are a contract analysis expert tasked with extracting structured information from vendor contract documents. Your primary goal is to identify master agreements and any related amendments that modify their expiry dates, so that the ultimate expiry date of each master agreement can be determined. From the provided contract text, extract the following fields in JSON format:
          
          ## INFO TO EXTRACT
          AgreementName: The name of the agreement, often followed by “(the Agreement)”.
          
          AgreementType: Classify the agreement using this guide (use filename as a supportive clue only, never a substitute for reading and understanding the document content):
          CC = Construction Contract
          NDA = Non-Disclosure Agreement
          CA = Confidentiality Agreement
          MWA = Miscellaneous Work Agreement
          MSA = Master Supply Agreement
          MSSA = Master Supply and Service Agreement
          EP = Engineering and Procurement Contract
          EPC = Engineering Procurement and Construction Contract
          T&C = Terms and Conditions
          CSA = Consulting Services Agreement
          MOU = Memorandum of Understanding
          MOA = Memorandum of Agreement
          OTHER = Not covered above
          NONAGREEMENT = Not an agreement
          
          If the filename contains tokens like “CA”, “NDA”, “MSA”, “MSSA”, “EPC”, “EP”, “CC”, “CSA”, or “Terms”, it may indicate the AgreementType. Always confirm with document content.
          
          DocumentType: Classify the document type as --
          AGREEMENT
          AMENDMENT (including Change Orders, CCO, COR)
          Rate Tables / Rate Schedules
          Collective Bargaining Agreements
          Contract Executive Summary
          CRAF (Recommendation for Award)
          Proposals
          Technical Drawings
          Other
          
          EffectiveDate: The start or effective date of the contract.
          
          ExpiryDate: The end or expiry date of the contract. If the TERM section specifies duration (e.g., “3 years from Effective Date”), calculate the end date.
          
          IsMasterAgreement: Boolean (true/false) - Is this document a master agreement?
          
          RelatedMasterAgreementName: If this is an amendment, specify the name of the master agreement it modifies.
          
          AmendmentExpiryDate: If this is an amendment, extract the new expiry date of the master agreement being amended, not the expiry date of the amendment document itself.
          
          ## ADDITIONAL INSTRUCTIONS
          Vendor Paths:
          In most cases, amendment documents are located in the same folder as the master agreement they modify. If a document is determined to be an amendment, there is a very high likelihood that the master agreement it amends is in the same folder. Use the provided file names from the vendor folder as a strong clue when linking amendments to their master agreements, but still confirm using document content (e.g., references to agreement name or number).
          
          Agreements and References:
          The document will likely contain both agreement and other reference names. These have already been extracted and are provided as references, but may not be complete. Use this as additional context.

          Special Rules for Expiry Dates:
          If the document is a master agreement, record its original expiry date in ExpiryDate.
          If the document is an amendment, record the new master agreement expiry in AmendmentExpiryDate.
          If multiple amendments exist, the ultimate expiry date for the master agreement will be the latest date among all amendments.
          
          Output Format:
          Return the result in this JSON structure:
          {
            "agreement_name": "",
            "agreement_type": "",
            "document_type": "",
            "effective_date": "",
            "expiry_date": "",
            "is_master_agreement": "",
            "related_master_agreement_name": "",
            "amendment_expiry_date": ""
          }
          
          If you are not confident about a field, return an empty string ("").

          CONTRACT INFORMATION',
          'Vendor Name:', vendor_name, '\n',
          'File Name:', file_name, '\n',
          'Other Vendor Paths', array_join(vendor_folder_paths, ', '), '\n',
          'Extracted References', combined_references, '\n'
          'Text:', truncated, '\n'
        ),1,:max_input_char),
        responseFormat => 'STRUCT<result:STRUCT<
          agreement_name:STRING, 
          agreement_type:STRING, 
          document_type:STRING, 
          effective_date:STRING, 
          expiry_date:STRING, 
          is_master_agreement:STRING, 
          related_master_agreement_name:STRING,
          amendment_expiry_date:STRING
          >>'
      ),
      'STRUCT<
          agreement_name:STRING, 
          agreement_type:STRING, 
          document_type:STRING, 
          effective_date:STRING, 
          expiry_date:STRING, 
          is_master_agreement:STRING, 
          related_master_agreement_name:STRING,
          amendment_expiry_date:STRING
        >'
      )
      as metadata
    FROM IDENTIFIER(:catalog || '.' || :schema || '.flat') f
    LEFT JOIN IDENTIFIER(:catalog || '.' || :schema || '.references') r
      ON f.path = r.path
    ANTI JOIN IDENTIFIER(:catalog || '.' || :schema || '.doc_info') d
      ON f.path = d.path
    LIMIT CAST(:batch_size AS INT)
  )
) AS source
ON target.path = source.path
WHEN NOT MATCHED THEN INSERT *;